In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd ..
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/
bin	 dev   lib32   mnt			 python-apt  srv    usr
boot	 etc   lib64   NGC-DL-CONTAINER-LICENSE  root	     sys    var
content  home  libx32  opt			 run	     tmp
datalab  lib   media   proc			 sbin	     tools


In [5]:
from matplotlib import pyplot as plt
import numpy as np
import os
from PIL import Image
import binascii

dir = 'content/drive/MyDrive/stegno'
cover = 'content/drive/MyDrive/stegno/Cover/cover_img.jpg'
secret = 'content/drive/MyDrive/stegno/Secret/5096.jpg'
out = 'content/drive/MyDrive/stegno/Out/stego_out.png'
'''
print("Do you want to set paths (y/n):")
opt = input()
if opt == 'y':
  print("Do you want to set cover image path (y/n):")
  opt = input()
  if opt == 'y':
    cover = input()
  print("Do you want to set Secret image path (y/n):")
  opt = input()
  if opt == 'y':
    secret = input()
  print("Do you want to set output image path (y/n):")
  opt = input()
  if opt == 'y':
    out = input()'''

'\nprint("Do you want to set paths (y/n):")\nopt = input()\nif opt == \'y\':\n  print("Do you want to set cover image path (y/n):")\n  opt = input()\n  if opt == \'y\':\n    cover = input()\n  print("Do you want to set Secret image path (y/n):")\n  opt = input()\n  if opt == \'y\':\n    secret = input()\n  print("Do you want to set output image path (y/n):")\n  opt = input()\n  if opt == \'y\':\n    out = input()'

In [19]:
import binascii
#print("Enter text to hide")
#txt = input()
txt = "0hello world!!!"
bin_str = ''
text = []

for i in txt:
  byt = bytes(i,'utf-8')
  BINARY = bin(int(binascii.hexlify(byt),16))
  binn = BINARY[2:]
  while len(binn)<8:
    binn = '0' + binn
  bin_str+= binn
  text.append(binn)

In [20]:
#pre process image
img = Image.open(cover).convert('L')
length, width = img.size
map = img.load()

stego_img = np.zeros([length, width], dtype=int)

file = open('content/drive/MyDrive/stegno/binary.txt','w')
img = []
for i in range(0,length):
  for j in range(0,width):
    pixel = map[i,j]
    stego_img[i][j] = pixel
    BINARY = bin(pixel)
    binn = BINARY[2:]
    while len(binn) < 8:
      binn = "0" + binn
    binn = binn[:7] +'0'
    img.append(binn)
file.close()

In [21]:
#Parity checker
# Expects atleast 7 bit long binary string
def byte_is_odd_parity(img):
  if (int(img[0]) + int(img[1]) + int(img[2]) + int(img[3]) + int(img[4]) + int(img[5]) + int(img[6]) ) % 2 == 1:
    return 1
  return 0

In [22]:
#To test if parity checker is working
print(byte_is_odd_parity(img[1]))

0


In [23]:

#Encoding
#check parity and split functionalities
t_ctr=2
row=0
col=0
for i in range(0,len(img)):
  t_seg = bin_str[t_ctr-2:t_ctr]
  if byte_is_odd_parity(img[i]):
    Msb = int(img[i][0]) & 0
    Msb1= int(img[i][1]) & 1
    #print(i,"th, odd",Msb,Msb1)
    if (str(Msb)+str(Msb1)) == t_seg:
      img[i] = img[i][0:7] + '1'
      #print(img[i]," ", t_seg)
      t_ctr+=2
  else:
    Msb = int(img[i][0]) | 1
    Msb1= int(img[i][1]) & 1
    #print(i,"th, even",Msb,Msb1)
    if (str(Msb)+str(Msb1)) == t_seg:
      img[i] = img[i][0:7] + '1'
      #print(img[i]," ", t_seg)
      t_ctr+=2
  stego_img[row][col] = int(img[i],2)
  col+=1
  if col == width:
    col = 0
    row+= 1

In [24]:
#Save encoded image
im = Image.fromarray((stego_img.transpose()).astype(np.uint8))
im.save(out)

In [25]:

encoded = 'content/drive/MyDrive/stegno/Out/stego_out.png'

enc_img = Image.open(encoded).convert('L')
length, width = enc_img.size
map = enc_img.load()
print(length, width, enc_img.mode)


secret_text = ""
for i in range(0,length):
  for j in range(0,width):
    pixel = map[i,j]
    BINARY = bin(pixel)
    binn = BINARY[2:]
    while len(binn) < 8:
      binn = "0" + binn
    if binn[-1] == '1':
      if byte_is_odd_parity(binn):
        Msb = str(int(binn[0]) & 0)
        Msb1= str(int(binn[1]) & 1)
        secret_text = secret_text + (Msb+Msb1)
      else:
        Msb = str(int(binn[0]) | 1)
        Msb1= str(int(binn[1]) & 1)
        secret_text = secret_text + (Msb+Msb1)

file = open('content/drive/MyDrive/stegno/extracted secret.txt','w')
for i in range(0,len(secret_text),8):
  byte_string = secret_text[i:i+8]
  character = chr(int(byte_string,2))
  print(character,byte_string)
  file.write(character)
file.close()

800 524 L
0 00110000
h 01101000
e 01100101
l 01101100
l 01101100
o 01101111
  00100000
w 01110111
o 01101111
r 01110010
l 01101100
d 01100100
! 00100001
! 00100001
! 00100001
